In [ ]:
import csv
import time
import math
import heapq
from collections import deque, defaultdict
from google.colab import files
import pandas as pd
from IPython.display import display

uploaded = files.upload()

def find_upload_files(target): # Goes through each uploaded file
    for name in uploaded.keys():
        if target.split('.')[0] in name:
            return name
    return None

def load_node_coordinates(filename): # Opens node files as CSV
    coordinates = {}
    with open(filename, 'r') as file:
        reader = csv.reader(file)
        for row in reader:
            node, x, y = row
            coordinates[node] = (float(x), float(y))
    return coordinates

def load_edge_list(filename): # Opens edge list files as TXT
    edges = defaultdict(list)
    with open(filename, 'r') as file:
        for line in file:
            if line.strip():
                n1, n2, w = line.split(',')
                edges[n1].append((n2, float(w)))
                edges[n2].append((n1, float(w)))
    return edges

def bfs(start, end, edges): # Breadth-First Search (uninformed search)
    order = []
    visited = set()
    queue = deque([start])
    visited.add(start)
    while queue:
        node = queue.popleft()
        order.append(node)
        if node == end:
            break
        for neighbor, _ in edges[node]:
            if neighbor not in visited:
                visited.add(neighbor)
                queue.append(neighbor)
    return order

def dfs(start, end, edges): # Depth-First Search (uninformed search)
    order = []
    visited = set()
    stack = [start]
    while stack:
        node = stack.pop()
        if node not in visited:
            visited.add(node)
            order.append(node)
            if node == end:
                break
            for neighbor, _ in edges[node]:
                if neighbor not in visited:
                    stack.append(neighbor)
    return order

def euclidean(a, b): # Heuristic function #1 (linear-based distance)
    return math.sqrt((a[0] - b[0]) ** 2 + (a[1] - b[1]) ** 2) # Equation

def manhattan(a, b): # Heuristic function #2 (grid-based distance)
    return abs(a[0] - b[0]) + abs(a[1] - b[1]) # Equation

def a_star(start, end, edges, coordinates, heuristic_fn): # A* Algorithm (informed search)
    visited = set()
    open_set = []
    heapq.heappush(open_set, (0, start, [start], 0))
    while open_set:
        f, node, path, total_cost = heapq.heappop(open_set)
        if node == end:
            return path
        if node in visited:
            continue
        visited.add(node)
        for neighbor, cost in edges[node]:
            if neighbor not in visited:
                new_cost = total_cost + cost
                h = heuristic_fn(coordinates[neighbor], coordinates[end])
                new_f = new_cost + h
                new_path = path + [neighbor]
                heapq.heappush(open_set, (new_f, neighbor, new_path, new_cost))
    return None

testcases = {
    "01": {"node": "TestCase_01_NodeID.csv", "edge": "TestCase_01_EdgeList.txt"},
    "02": {"node": "TestCase_02_NodeID.csv", "edge": "TestCase_02_EdgeList.txt"},
    "03": {"node": "TestCase_03_NodeID.csv", "edge": "TestCase_03_EdgeList.txt"},
}

results = []

for caseid, fileset in testcases.items():

        node_file = find_upload_files(fileset["node"]) # Finds uploaded node file (CSV)
        edge_file = find_upload_files(fileset["edge"]) # Finds uploaded edge list file (TXT)

        if not node_file or not edge_file:
            print(f"\nSkipping test case {caseid}: File(s) not found: {node_file}, {edge_file}")
            continue

        coordinates = load_node_coordinates(node_file)
        edges = load_edge_list(edge_file)
        start = list(coordinates.keys())[0] # Start = first node
        end = list(coordinates.keys())[-1] # End = last node

        # Records the timing for each algorithm
        time0 = time.time()
        path_bfs = bfs(start, end, edges)
        time1 = time.time()
        path_dfs = dfs(start, end, edges)
        time2 = time.time()
        path_euclidean = a_star(start, end, edges, coordinates, euclidean)
        time3 = time.time()
        path_manhattan = a_star(start, end, edges, coordinates, manhattan)
        time4 = time.time()
        combined_time_bfs = time1 - time0
        combined_time_dfs = time2 - time1
        combined_time_euclidean = time3 - time2
        combined_time_manhattan = time4 - time3

        results.append({"Test Case": caseid, "Algorithm": "BFS", "Steps": len(path_bfs), "Time (seconds)": f"{combined_time_bfs:.9f}"})
        results.append({"Test Case": caseid, "Algorithm": "DFS", "Steps": len(path_dfs), "Time (seconds)": f"{combined_time_dfs:.9f}"})
        results.append({"Test Case": caseid, "Algorithm": "A* (Euclidean)", "Steps": len(path_euclidean), "Time (seconds)": f"{combined_time_euclidean:.9f}"})
        results.append({"Test Case": caseid, "Algorithm": "A* (Manhattan)", "Steps": len(path_manhattan), "Time (seconds)": f"{combined_time_manhattan:.9f}"})

        # Prints the output for each algorithm
        print(f"\n----- Test Case {caseid} -----")
        print(f"BFS Path:", path_bfs)
        print(f"DFS Path:", path_dfs)
        print(f"A* Path (Euclidean Heuristic):", path_euclidean)
        print(f"A* Path (Manhattan Heuristic):", path_manhattan)

print(f"\n---------- Assignment 1 Summary Table ----------")
data_frame = pd.DataFrame(results)
data_frame_style = data_frame.style.set_properties(**{'text-align': 'left'}).hide(axis="index")
display(data_frame_style)


# Summary of Assignment 1 - Using Search for Robot Path Planning

# For this project, I decided to implement three search algorithms: Breadth-First Search (BFS), Depth-First Search (DFS), and A*.
# BFS is an uninformed search that uses a queue (deque) to explore different neighbors, DFS is an uninformed search that uses a stack to extensively explore paths,
# and A* is an informed search that uses a heap queue (Python's heapq) and heuristic functions to effectively get the search towards its end goal.
# For the A* algorithm, I decided to implement two heuristics: Euclidean and Manhattan. Euclidean is typically based on linear distance, while Manhattan is typically based on grid distance.
# The metrics of choice that I used to compare the performance of these three algorithms are the number of steps taken and execution time for each path.
# According to the results of each test case, the A* Path took the shortest amount of steps, while the DFS Path took the shortest amount of time.
# BFS takes a shorter path than the DFS but is not the most reliable in larger graphs, while DFS often takes a longer path and is less optimized than BFS.
# Overall, the most suitable algorithm for this problem is A*. It is more reliable and uses an informed search that is based on domain knowledge of the best paths for these searches.